In [ ]:
from setup_triton import setup_triton

# TRITON_INTERPRET=1 uses a python interpreter instead of running on the GPU. 
# This menas that uou can insert Python breakpoints to debug your kernel code! 
setup_triton(use_interpreter=True)

# Triton Puzzle 7: Softmax Attention

Welcome to the last Triton puzzle! Attention is the cornerstone of modern transformers. This puzzle teaches you how to implement the (forward pass of the) scaled dot-product attention mechanism efficiently.

### What you'll learn:
- **Multi-stage computation** with multiple matrix operations
- **2D softmax** across sequence dimension
- **Causal masking** for autoregressive models
- **Memory-efficient attention** for long sequences
- **Numerical stability** with proper scaling
- Introduction to the _main_ ideas behind **Flash Attention**

## Mathematical Background

Scaled dot-product attention computes:

$$
\text{Attention}(Q, K, V) = \text{softmax}\left(Q K^\top / \sqrt{d} \right) V
$$

Where: $Q, K, V \in \mathbb{R}^{B \times H \times N \times d}$, with B = batch size, H = number of heads, N = sequence length, d = head size.

### The Challenge

Standard attention has $O(N^2)$ memory complexity:
1. Compute scores: $S = Q K^\top$, a (B x H x N x N) tensor
2. Apply softmax: $P = \text{softmax}(S)$, still (B x H x N x N)
3. Compute output: $O = PV$, with (B x H x N x d_v)

For long sequences, the N x N attention matrix becomes prohibitive!


### Flash Attention Insight

Instead of materializing the full N x N attention matrix:
- Process attention in blocks
- Recompute rather than store intermediate values
- Use online softmax for numerical stability and faster computation

💡 Final Tip: Remember that you can use `TRITON_INTERPRET=1` to debug your kernels.

In [ ]:
import torch
import triton
import triton.language as tl
import numpy as np
from IPython.display import display, Image

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {DEVICE}")

# Set random seed
torch.manual_seed(42)

## Implementation 1: Naive PyTorch

First, let's see the standard implementation:

In [ ]:
def attention_naive(Q, K, V, causal=False):
    """
    Naive attention implementation.
    Creates the full N×N attention matrix.
    
    Args:
        Q, K, V: (B, H, N, d) tensors
        causal: whether to apply causal mask
    """
    B, H, N, d = Q.shape
    
    # Compute attention scores
    scores = torch.matmul(Q, K.transpose(-2, -1)) / (d ** 0.5)  # (B, H, N, N)
    
    # Apply causal mask if needed
    if causal:
        mask = torch.triu(torch.ones(N, N, device=Q.device), diagonal=1).bool()
        scores.masked_fill_(mask[None, None, :, :], float('-inf'))
    
    # Apply softmax
    attn_weights = torch.softmax(scores, dim=-1)  # (B, H, N, N)
    
    # Compute output
    out = torch.matmul(attn_weights, V)  # (B, H, N, d)
    
    return out

## Implementation 2: PyTorch Built-in

PyTorch's scaled_dot_product_attention (introduced in PyTorch 2.0):

In [ ]:
def attention_pytorch(Q, K, V, causal=False):
    """PyTorch's built-in attention (uses Flash Attention when possible)."""
    # Note: PyTorch expects (B, H, N, d) format
    return torch.nn.functional.scaled_dot_product_attention(
        Q, K, V, 
        is_causal=causal,
        dropout_p=0.0
    )

## Implementation 3: PyTorch Compiled



In [ ]:
@torch.compile
def attention_compiled(Q, K, V, causal=False):
    """Compiled attention."""
    B, H, N, d = Q.shape
    scores = torch.matmul(Q, K.transpose(-2, -1)) / (d ** 0.5)
    if causal:
        mask = torch.triu(torch.ones(N, N, device=Q.device), diagonal=1).bool()
        scores.masked_fill_(mask, float('-inf'))
    attn_weights = torch.softmax(scores, dim=-1)
    out = torch.matmul(attn_weights, V)
    return out

## Key Concepts for This Puzzle

### 1. Block Processing Strategy

Instead of computing the full attention matrix:
```python
# Naive: O(N^2) memory
scores = Q @ K.T  # N×N matrix

# Block-wise: O(N) memory
for block_q in Q_blocks:
    for block_k in K_blocks:
        block_scores = block_q @ block_k.T  # Bq x Bk matrix
        # Process block...
```

### 2. Online Softmax for 2D

Extending our 1D online softmax to 2D:
- Each query attends to all keys
- Need to track max and sum for each query
- Update statistics as we process key blocks

### 3. Causal Masking

For autoregressive models:
- Query at position _i_ can only attend to positions _[1, ..., i]_
- Implement efficiently without creating full mask matrix

### 4. Memory vs Recomputation Trade-off

- Standard: Store N x N attention matrix
- Memory-efficient: Recompute blocks as needed
- Flash Attention: Never materialize full attention matrix

## Implementation 4: Triton Kernel (Puzzle)

Now implement memory-efficient attention!

We'll start with a simpler version that still materializes attention weights (but processes in blocks), then show the full Flash Attention style approach.

### Version 1: Block-wise Attention (Starter)

In [ ]:
@triton.jit
def attention_kernel_v1(
    Q_ptr, K_ptr, V_ptr, Out_ptr,
    B, H, N, d,
    scale,
    stride_qb, stride_qh, stride_qn, stride_qd,
    stride_kb, stride_kh, stride_kn, stride_kd,
    stride_vb, stride_vh, stride_vn, stride_vd,
    stride_ob, stride_oh, stride_on, stride_od,
    causal: tl.constexpr,
    BLOCK_N: tl.constexpr,
    BLOCK_D: tl.constexpr,
):
    """
    Simplified attention kernel - still computes full attention per query.
    Each program handles one (batch, head, query) combination.
    
    This version is easier to understand but less memory efficient.
    """
    # YOUR IMPLEMENTATION GOES HERE
    # Hints:
    # 1. Get batch, head, and query indices from program ID
    # 2. Load the query vector (size d)
    # 3. Loop over all keys to compute attention scores
    # 4. Apply softmax to get weights
    # 5. Use weights to compute weighted sum of values
    pass


def attention_triton_v1(Q, K, V, causal=False):
    """Wrapper for Triton attention kernel."""
    assert Q.shape == K.shape
    assert K.shape[-1] == V.shape[-1]
    B, H, N, d = Q.shape
    scale = 1.0 / (d**0.5)
    
    # Allocate output
    Out = torch.empty_like(Q).contiguous()

    # Ensure contiguity
    Q = Q.contiguous()
    K = K.contiguous()
    V = V.contiguous()
    
    # Choose block sizes
    BLOCK_N = min(triton.next_power_of_2(N), 64)
    BLOCK_D = d  # = d for simplicity
    
    # Launch kernel (one program per batch, head, query)
    grid = (B * H * N,)
    
    attention_kernel_v1[grid](
        Q, K, V, Out,
        B, H, N, d,
        scale,
        Q.stride(0), Q.stride(1), Q.stride(2), Q.stride(3),
        K.stride(0), K.stride(1), K.stride(2), K.stride(3),
        V.stride(0), V.stride(1), V.stride(2), V.stride(3),
        Out.stride(0), Out.stride(1), Out.stride(2), Out.stride(3),
        causal,
        BLOCK_N, 
        BLOCK_D,
    )
    
    return Out

### Version 2: Flash Attention Style (Advanced)

In [ ]:
@triton.jit
def attention_kernel_v2(
    Q_ptr, K_ptr, V_ptr, Out_ptr,
    B, H, N, d,
    scale,
    stride_qb, stride_qh, stride_qn, stride_qd,
    stride_kb, stride_kh, stride_kn, stride_kd,
    stride_vb, stride_vh, stride_vn, stride_vd,
    stride_ob, stride_oh, stride_on, stride_od,
    causal: tl.constexpr,
    BLOCK_M: tl.constexpr,  # Block size for queries
    BLOCK_N: tl.constexpr,  # Block size for keys/values
    BLOCK_D: tl.constexpr,  # Block size for head dimension
):
    """
    Flash Attention style kernel - never materializes full attention.
    Each program handles a block of queries.
    
    Key insight: Use online softmax to avoid storing attention weights!
    """
    # YOUR IMPLEMENTATION GOES HERE
    # This is more complex - see Algorithm 1 in https://arxiv.org/pdf/2307.08691
    pass


def attention_triton_v2(Q, K, V, causal=False):
    """Flash Attention style implementation."""
    assert Q.shape == K.shape
    assert K.shape[-1] == V.shape[-1]
    B, H, N, d = Q.shape
    scale = 1.0 / (d**0.5)

    # Allocate output
    Out = torch.empty_like(Q).contiguous()

    # Ensure contiguity
    Q = Q.contiguous()
    K = K.contiguous()
    V = V.contiguous()
    
    # Block sizes
    BLOCK_M = min(triton.next_power_of_2(N), 64)
    BLOCK_N = min(triton.next_power_of_2(N), 64)
    BLOCK_D = d  # = d for simplicity
    
    grid = (triton.cdiv(N, BLOCK_M), B * H)
    
    attention_kernel_v2[grid](
        Q, K, V, Out,
        B, H, N, d,
        scale,
        Q.stride(0), Q.stride(1), Q.stride(2), Q.stride(3),
        K.stride(0), K.stride(1), K.stride(2), K.stride(3),
        V.stride(0), V.stride(1), V.stride(2), V.stride(3),
        Out.stride(0), Out.stride(1), Out.stride(2), Out.stride(3),
        causal,
        BLOCK_M, 
        BLOCK_N, 
        BLOCK_D,
    )

    return Out

## Solution 🧙

You shall not pass!

### Version 1: Simple Block-wise Solution

In [ ]:
# Our implementation goes here

### Version 2: Flash Attention Style (Advanced)

In [ ]:
# Our implementation goes here

## Testing Correctness

### Version 1

In [ ]:
def test_attention(B=2, H=4, N=128, d=64, causal=False, atol=1e-2, rtol=1e-2):
    """Test attention implementation against PyTorch."""
    torch.manual_seed(42)
    
    # Generate random inputs
    Q = torch.randn(B, H, N, d, device=DEVICE, dtype=torch.float32)
    K = torch.randn(B, H, N, d, device=DEVICE, dtype=torch.float32)
    V = torch.randn(B, H, N, d, device=DEVICE, dtype=torch.float32)
    
    # PyTorch reference
    expected = attention_naive(Q, K, V, causal=causal)
    
    # Triton implementation
    actual = attention_triton_v1(Q, K, V, causal=causal)
    
    try:
        torch.testing.assert_close(actual, expected, atol=atol, rtol=rtol)
        print(f"✅ Test PASSED! ({'causal' if causal else 'non-causal'})")
        print(f"   Shape: (B={B}, H={H}, N={N}, d={d})")
        print(f"   Max error: {(actual - expected).abs().max().item():.2e}")
        return True
    except AssertionError as e:
        print(f"❌ Test FAILED!")
        print(f"   Error: {e}")
        return False

# Test both causal and non-causal
test_passed = test_attention(causal=False)
test_passed &= test_attention(causal=True)

# Display congrats message
if test_passed:
    print("\n🎉 Congratulations! Your implementation is correct!")
    display(Image("figs/success.gif", width=256, height=256))

### Version 2

In [ ]:
def test_attention(B=2, H=4, N=128, d=64, causal=False, atol=1e-2, rtol=1e-2):
    """Test attention implementation against PyTorch."""
    torch.manual_seed(42)
    
    # Generate random inputs
    Q = torch.randn(B, H, N, d, device=DEVICE, dtype=torch.float32)
    K = torch.randn(B, H, N, d, device=DEVICE, dtype=torch.float32)
    V = torch.randn(B, H, N, d, device=DEVICE, dtype=torch.float32)
    
    # PyTorch reference
    expected = attention_naive(Q, K, V, causal=causal)
    
    # Triton implementation
    actual = attention_triton_v2(Q, K, V, causal=causal)
    
    try:
        torch.testing.assert_close(actual, expected, atol=atol, rtol=rtol)
        print(f"✅ Test PASSED! ({'causal' if causal else 'non-causal'})")
        print(f"   Shape: (B={B}, H={H}, N={N}, d={d})")
        print(f"   Max error: {(actual - expected).abs().max().item():.2e}")
        return True
    except AssertionError as e:
        print(f"❌ Test FAILED!")
        print(f"   Error: {e}")
        return False

# Test both causal and non-causal
test_passed = test_attention(causal=False)
test_passed &= test_attention(causal=True)

# Display congrats message
if test_passed:
    print("\n🎉 Congratulations! Your implementation is correct!")
    display(Image("figs/success.gif", width=256, height=256))

## FLOP and Memory Analysis

In [ ]:
def analyze_attention(B, H, N, d):
    """Analyze memory and compute for attention."""
    element_size = 4  # float32
    
    # Standard attention
    standard = {
        'attention_matrix': B * H * N * N * element_size,  # Q @ K^T
        'total_memory': B * H * N * N * element_size + B * H * N * d * element_size,
    }
    
    # Flash attention (approximate)
    block_size = 128
    flash = {
        'working_memory': B * H * block_size * block_size * element_size,  # One block at a time
        'total_memory': B * H * block_size * block_size * element_size,
    }
    
    # Compute
    flops = B * H * (2 * N * N * d + 2 * N * N * d)  # Q@K^T + softmax@V
    
    return {
        'standard_memory': standard['total_memory'],
        'flash_memory': flash['total_memory'],
        'memory_reduction': standard['total_memory'] / flash['total_memory'],
        'flops': flops,
    }

# Example
B, H, N, d = 8, 16, 512, 64
analysis = analyze_attention(B, H, N, d)

print(f"Attention Analysis (B={B}, H={H}, N={N}, d={d}):")
print(f"  Standard memory: {analysis['standard_memory'] / 1e9:.2f} GB")
print(f"  Flash memory: {analysis['flash_memory'] / 1e6:.2f} MB")
print(f"  Memory reduction: {analysis['memory_reduction']:.0f}x")
print(f"  FLOPs: {analysis['flops'] / 1e9:.2f} GFLOPs")

## Summary

You've successfully implemented the (forward pass) of Flash Attention - one of the cornerstone of modern AI!


### Performance Insights:

- **Memory bottleneck**: O(N²) attention matrix is the main constraint
- **Flash Attention**: Never materializing full attention saves massive memory
- **Recomputation**: Trading compute for memory is often worthwhile
- **Block sizes**: Critical for balancing parallelism and memory usage

### Real-World Impact:

- **Longer sequences**: Flash Attention enables processing much longer sequences
- **Larger models**: Memory savings allow bigger batch sizes
- **Energy efficiency**: Less memory movement = less energy consumption
- **Production ready**: These techniques power ChatGPT, Claude, and other LLMs!

### Advanced Extensions:

- **Multi-Query Attention (MQA)**: Share keys/values across heads
- **Grouped-Query Attention (GQA)**: Share keys/values across groups of heads
- **Sliding Window Attention**: Local attention patterns
- **Sparse Attention**: Attend to specific positions only


<center>
    <b><span style="font-size: 24px">Flash Swim, Sardine!</span></b>
    <br>
    <img src="figs/sardine-super-swim.png" width="512" />
</center>

---

## Benchmarking (GPU only)

In [ ]:
@triton.testing.perf_report(
    triton.testing.Benchmark(
        x_names=['N'],  # Sequence length
        x_vals=[128, 256, 512, 1024, 2048, 4096],  #, 8192, 16384, 32768, 65536],
        line_arg='provider',
        line_vals=['naive', 'pytorch', 'compiled', 'triton_v1', 'triton_v2'],
        line_names=['Naive', 'PyTorch', 'Compiled', 'Triton v1', 'Triton v2'],
        styles=[('blue', '-'), ('green', '--'), ('orange', '-.'), ('red', ':'), ('black', ':')],
        ylabel='TFLOPS',
        plot_name='attention-performance',
        args={'B': 4, 'H': 8, 'd': 64, 'causal': True},  # feel free to edit these values!
    )
)
def benchmark(N, B, H, d, causal, provider):
    """Benchmark attention implementations."""
    Q = torch.randn(B, H, N, d, device=DEVICE, dtype=torch.float32)
    K = torch.randn(B, H, N, d, device=DEVICE, dtype=torch.float32)
    V = torch.randn(B, H, N, d, device=DEVICE, dtype=torch.float32)
    
    # Normalize
    Q = Q / Q.norm(dim=-1, keepdim=True)
    K = K / K.norm(dim=-1, keepdim=True)
    
    quantiles = [0.5, 0.05, 0.95]
    
    if provider == 'naive':
        ms, min_ms, max_ms = triton.testing.do_bench(
            lambda: attention_naive(Q, K, V, causal=causal), quantiles=quantiles
        )
    elif provider == 'pytorch':
        ms, min_ms, max_ms = triton.testing.do_bench(
            lambda: attention_pytorch(Q, K, V, causal=causal), quantiles=quantiles
        )
    elif provider == 'compiled':
        ms, min_ms, max_ms = triton.testing.do_bench(
            lambda: attention_compiled(Q, K, V, causal=causal), quantiles=quantiles
        )
    elif provider == 'triton_v1':
        ms, min_ms, max_ms = triton.testing.do_bench(
            lambda: attention_triton_v1(Q, K, V, causal=causal), quantiles=quantiles
        )
    elif provider == 'triton_v2':
        ms, min_ms, max_ms = triton.testing.do_bench(
            lambda: attention_triton_v2(Q, K, V, causal=causal), quantiles=quantiles
        )
    
    # Calculate FLOPS (approximate)
    flops = B * H * (2 * N * N * d + 2 * N * N * d)
    tflops = lambda ms: flops / ms / 1e9
    
    return tflops(ms), tflops(max_ms), tflops(min_ms)

print("Running benchmarks...")
results = benchmark.run(show_plots=True, print_data=True, return_df=True, save_path='')

## Speedup?

In [ ]:
# Check if Triton is faster than PyTorch
avg_pytorch = results['PyTorch'].mean()
avg_triton = results['Triton'].mean()
speedup = avg_triton / avg_pytorch

if speedup > 1.0:
    print(f"\n🚀 Awesome! Triton is {speedup:.2f}x faster than PyTorch!")
    display(Image("figs/gpu.gif", width=400, height=256))
else:
    print(f"\n🐌 Not quite yet! Triton implementation is {speedup:.2f}x slower than PyTorch!")